In [295]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score, KFold

import matplotlib.pyplot as plt

import os
import re

from collections import Counter

In [332]:
import plotly
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'notebook'
from plotly.subplots import make_subplots

In [296]:
from imblearn.over_sampling import SMOTE

# 合併資料並做標準化

In [4]:
球員資料 = os.getcwd() + '/球員資料'
f_df   = pd.DataFrame({})
for file in os.listdir(球員資料):
    if re.match(r'\d{4}',file):
        specific_file = 球員資料+'/'+file
        specific_df   = pd.read_csv(specific_file)
        col_not_need  = ['AGE','Year','Pos']
        specific_df   = specific_df[[col for col in specific_df.columns if col not in col_not_need]]
        specific_df['是否首次入選']     = specific_df['是否首次入選'].map({True:1,False:0})
        specific_df['當年有無入選']     = specific_df['當年有無入選'].map({True:1,False:0})
        specific_df                   = specific_df.dropna(subset = ['Final_Position'])
        specific_df.reset_index(drop=True,inplace=True)
        #specific_df['Final_Position'] = specific_df['Final_Position'].apply(lambda x: x[1] if re.match(r'[A-Z]{2}',str(x)) else x)
        position_dummy_df             = pd.get_dummies(specific_df.position)
        specific_df_final             = pd.concat([specific_df,position_dummy_df],axis=1)
        #specific_df_final.drop(columns=['Final_Position'],inplace=True)

        sc = StandardScaler()
        數值_columns   = ['GP', 'W', 'L', 'MIN', 'PTS', 'FGA', 'FG%', '3PA', '3P%', 'FTA', 'FT%',
        'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'PF', 'FP', 'DD2',
        'TD3', '+/-', '累積前三隊', '勝率', 'DFG_percent','DIFF_percent']
        非數值_columns = [col for col in specific_df_final.columns if col not in 數值_columns]
        print(非數值_columns)
        數值資料_df     = specific_df_final[數值_columns]
        非數值資料_df   = specific_df_final[非數值_columns]
        數值資料_df_std = pd.DataFrame(sc.fit_transform(數值資料_df),columns = 數值_columns)
        Final_DF       = pd.concat([數值資料_df_std,非數值資料_df],axis = 1)
        if re.match(r'\d{4}',file)[0] != '2022':
            print(re.match(r'\d{4}',file)[0])
            f_df          = f_df.append(Final_DF)

        file_path     = r'/Users/andyhsu/Desktop/NBA年度前三隊預測/建模資料_用position/' + file
        Final_DF.to_csv(file_path,index=False,header=True,encoding='UTF-8-SIG')

f_df.to_csv(r'/Users/andyhsu/Desktop/NBA年度前三隊預測/建模資料_用position/整合各年資料_改用position.csv',index=False,header=True,encoding='UTF-8-SIG')

['Player', 'Team', '當年有無入選', 'position', 'Final_Position', '是否首次入選', 'C', 'C-F', 'F', 'F-C', 'F-G', 'G', 'G-F']
2019
['Player', 'Team', '當年有無入選', 'position', 'Final_Position', '是否首次入選', 'C', 'C-F', 'F', 'F-C', 'F-G', 'G', 'G-F', 'PG']
2018
['Player', 'Team', '當年有無入選', 'position', 'Final_Position', '是否首次入選', 'C', 'C-F', 'F', 'F-C', 'F-G', 'G', 'G-F']
2014
['Player', 'Team', '當年有無入選', 'position', 'Final_Position', '是否首次入選', 'C', 'C-F', 'F', 'F-C', 'F-G', 'G', 'G-F']
['Player', 'Team', '當年有無入選', 'position', 'Final_Position', '是否首次入選', 'C', 'C-F', 'F', 'F-C', 'F-G', 'G', 'G-F']
2015
['Player', 'Team', '當年有無入選', 'position', 'Final_Position', '是否首次入選', 'C', 'C-F', 'F', 'F-C', 'F-G', 'G', 'G-F']
2021
['Player', 'Team', '當年有無入選', 'position', 'Final_Position', '是否首次入選', 'C', 'C-F', 'F', 'F-C', 'F-G', 'G', 'G-F']
2017
['Player', 'Team', '當年有無入選', 'position', 'Final_Position', '是否首次入選', 'C', 'C-F', 'F', 'F-C', 'F-G', 'G', 'G-F']
2016
['Player', 'Team', '當年有無入選', 'position', 'Final_Position', '是否首

In [506]:
final_data = pd.read_csv(r'/Users/andyhsu/Desktop/NBA年度前三隊預測/建模資料_用position/整合各年資料_改用position.csv',encoding='utf-8-sig')

In [507]:
len(final_data)

4021

In [508]:
final_data = final_data.dropna(subset=['position'])

In [509]:
len(final_data)

4021

In [510]:
final_data.columns

Index(['GP', 'W', 'L', 'MIN', 'PTS', 'FGA', 'FG%', '3PA', '3P%', 'FTA', 'FT%',
       'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'PF', 'FP', 'DD2',
       'TD3', '+/-', '累積前三隊', '勝率', 'DFG_percent', 'DIFF_percent', 'Player',
       'Team', '當年有無入選', 'position', 'Final_Position', '是否首次入選', 'C', 'C-F',
       'F', 'F-C', 'F-G', 'G', 'G-F', 'PG'],
      dtype='object')

In [511]:
資料 = ['GP', 'W', 'L', 'MIN', 'PTS', 'FGA', 'FG%', '3PA', '3P%', 'FTA', 'FT%','OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL',
        'BLK', 'PF', 'DD2', 'TD3', '+/-', '累積前三隊', '勝率', 'DFG_percent', 'DIFF_percent', 'C', 'C-F', 'F', 'F-C',
        'F-G', 'G', 'G-F','Player','Team','position','當年有無入選']

In [569]:
train_x,test_x,train_y,test_y = train_test_split(
                final_data[資料].iloc[:,:-4].values,
                final_data[資料].iloc[:,-1].values,
                test_size=.3,
                random_state=0,
                stratify=final_data[資料].iloc[:,-1].values
                )

lr   = LogisticRegression(
                penalty = 'l2',
                random_state = 2,
                class_weight= None
                )

print(len(train_x))
print(len(test_x))
'''X = train_x
Y = train_y
sm = SMOTE(random_state=42)
X_res, Y_res = sm.fit_resample(X,Y)
train_x      = X_res
train_y      = Y_res
print(len(train_x))'''

lr.fit(train_x,train_y)

2814
1207


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



LogisticRegression(random_state=2)

# Cross Validation

In [570]:
kf = KFold(n_splits=5)
cross_value = cross_val_score(lr,train_x,train_y,cv = kf)
cross_value

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/opt/anaconda3/lib/python3.8/site-packages

array([0.98934281, 0.98934281, 0.98756661, 0.99111901, 0.97864769])

# 看看成效

In [571]:
print('score:',lr.score(test_x,test_y))

score: 0.9908864954432477


In [572]:
print('lr.predict_proba:',lr.predict_proba(test_x))

lr.predict_proba: [[9.99999415e-01 5.84880372e-07]
 [9.92701841e-01 7.29815936e-03]
 [9.99999864e-01 1.35615764e-07]
 ...
 [1.00000000e+00 1.15776940e-10]
 [9.99977578e-01 2.24222338e-05]
 [1.00000000e+00 7.75479995e-11]]


# 自己做ROC AUC

In [573]:
def confusion_matrix(threshold, x_proba, test_y):
    final_result = []
    matrix = np.array([[0. , 0.],
                       [0. , 0.]])
    for x in x_proba:
        if x[1] > threshold:
            final_result.append(1)
        else:
            final_result.append(0)
            
    #test_y是測試集結果，final_result是預測結果
    for x in list(zip(test_y, final_result)):
        if x[0] == 0 and x[1] == 0:
            matrix[0][0] += 1
        elif x[0] == 0 and x[1] == 1:
            matrix[1][0] += 1
        elif x[0] == 1 and x[1] == 0:
            matrix[0][1] += 1
        else:
            matrix[1][1] += 1
    
    final_matrix = pd.DataFrame(matrix,
                        index   =pd.MultiIndex.from_tuples([('預測','沒入選'),('預測','入選')]),
                        columns = pd.MultiIndex.from_tuples([('實際','沒入選'),('實際','入選')]))
    print(pd.DataFrame(final_matrix))

    matrix_values = final_matrix.values
    Recall        = matrix_values[1][1]/(matrix_values[1][1]+matrix_values[0][1])
    if pd.isna(Recall):
        Recall = 0
    Specificity   = matrix_values[0][0]/(matrix_values[0][0]+matrix_values[1][0])
    if pd.isna(Specificity):
        Specificity = 0

    return Recall, 1-Specificity

In [574]:
all_index = []
for x in np.linspace(0,2,555):
    TPR, FPR = confusion_matrix(threshold = x, x_proba = lr.predict_proba(test_x),test_y=test_y)
    all_index.append([x, TPR, FPR])

            實際      
           沒入選    入選
預測 沒入選     0.0   0.0
   入選   1171.0  36.0
            實際      
           沒入選    入選
預測 沒入選  1100.0   0.0
   入選     71.0  36.0
            實際      
           沒入選    入選
預測 沒入選  1111.0   0.0
   入選     60.0  36.0
            實際      
           沒入選    入選
預測 沒入選  1117.0   0.0
   入選     54.0  36.0
            實際      
           沒入選    入選
預測 沒入選  1122.0   0.0
   入選     49.0  36.0
            實際      
           沒入選    入選
預測 沒入選  1128.0   0.0
   入選     43.0  36.0
            實際      
           沒入選    入選
預測 沒入選  1133.0   0.0
   入選     38.0  36.0
            實際      
           沒入選    入選
預測 沒入選  1134.0   0.0
   入選     37.0  36.0
            實際      
           沒入選    入選
預測 沒入選  1135.0   0.0
   入選     36.0  36.0
            實際      
           沒入選    入選
預測 沒入選  1137.0   0.0
   入選     34.0  36.0
            實際      
           沒入選    入選
預測 沒入選  1137.0   0.0
   入選     34.0  36.0
            實際      
           沒入選    入選
預測 沒入選  1137.0   0.0
   入選     34.

In [575]:
confusion_matrix(0.176895, lr.predict_proba(test_x), test_y)

            實際      
           沒入選    入選
預測 沒入選  1157.0   0.0
   入選     14.0  36.0


(1.0, 0.011955593509820672)

In [576]:
#Without_SMOTE
auc_data2 = pd.DataFrame(all_index)
auc_data2 = auc_data2.sort_values(by=0,ascending=True)
auc_data2

,0,1,2
0,0.00000,1.0,1.000000
1,0.00361,1.0,0.060632
2,0.00722,1.0,0.051238
3,0.01083,1.0,0.046114
4,0.01444,1.0,0.041845
...,...,...,...
550,1.98556,0.0,0.000000
551,1.98917,0.0,0.000000
552,1.99278,0.0,0.000000
553,1.99639,0.0,0.000000


In [577]:
auc_data2[(auc_data2[0]>=0.45)&(auc_data2[0]<=0.5)]

,0,1,2
125,0.451264,0.888889,0.00427
126,0.454874,0.888889,0.00427
127,0.458484,0.888889,0.00427
128,0.462094,0.888889,0.00427
129,0.465704,0.861111,0.00427
130,0.469314,0.861111,0.00427
131,0.472924,0.833333,0.00427
132,0.476534,0.833333,0.00427
133,0.480144,0.833333,0.00427
134,0.483755,0.833333,0.00427


In [518]:
#SMOTE
#auc_data = pd.DataFrame(all_index)
#auc_data = auc_data.sort_values(by=0,ascending=True)
#auc_data

,0,1,2
0,0.00000,1.0,1.000000
1,0.00361,1.0,0.031597
2,0.00722,1.0,0.030743
3,0.01083,1.0,0.030743
4,0.01444,1.0,0.029035
...,...,...,...
550,1.98556,0.0,0.000000
551,1.98917,0.0,0.000000
552,1.99278,0.0,0.000000
553,1.99639,0.0,0.000000


In [578]:
fig = go.Figure()



'''fig.add_trace(go.Scatter(
        x = auc_data[2],
        y = auc_data[1],
        mode    = 'lines',
        name    = 'ROC_Curve_SMOTE'
        ))'''

fig.add_trace(go.Scatter(
        x = auc_data2[2],
        y = auc_data2[1],
        mode    = 'lines',
        name    = 'ROC_Curve_Without_SMOTE',
        text = auc_data2[0],
        hovertemplate = ' Threshold: %{text:.2f}'
        ))


fig.update_layout(
        title = dict(
                text = 'Logistic Regression ROC_Curve',
                x    = 0.5,
                font = dict(
                        size = 25
                )
        ),
        xaxis=dict(
                title= 'FPR',
                showgrid = False
                ),
        yaxis=dict(
                title= 'TPR',
                showgrid = False
                ),
        paper_bgcolor='rgb(243, 243, 233)',
        plot_bgcolor='rgb(243, 243, 233)'
        )

fig.add_annotation(x=1, y=1,
            text="Threshold:0",
            showarrow=False,
            yshift=10)
        
fig.add_annotation(x=0, y=0,
            text="Threshold:2",
            showarrow=False,
            yshift=10)

fig.add_annotation(x=0.005978, y=0.777778,
            text="Threshold:0.5",
            showarrow=True,
            xref = 'x',
            yref = 'y',
            yshift=0,
            xshift=0,
            ax = 60,
            ay = 40)

fig.add_annotation(x=0.012810, y=auc_data2[1].max(),
            text= "Threshold:0.17",
            showarrow=False,
            yshift=10)

fig.add_trace(go.Scatter(x=auc_data2[2], y=[0] * len(auc_data2[2]), fill='tonexty',line_color='skyblue',name='AUC'))

fig.show()

In [579]:
f1_score(test_y, lr.predict(test_x), average='macro')

0.92018779342723

In [552]:
data_2022 = pd.read_csv('/Users/andyhsu/Desktop/NBA年度前三隊預測/建模資料_用position/2022球員資料.csv',encoding='utf-8-sig')
data_2022.columns

Index(['GP', 'W', 'L', 'MIN', 'PTS', 'FGA', 'FG%', '3PA', '3P%', 'FTA', 'FT%',
       'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'PF', 'FP', 'DD2',
       'TD3', '+/-', '累積前三隊', '勝率', 'DFG_percent', 'DIFF_percent', 'Player',
       'Team', '當年有無入選', 'position', 'Final_Position', '是否首次入選', 'C', 'C-F',
       'F', 'F-C', 'F-G', 'G', 'G-F'],
      dtype='object')

In [553]:
x_2022  = data_2022[資料].iloc[:,:-4]
lr.predict_proba(x_2022.values)

array([[9.66452703e-04, 9.99033547e-01],
       [1.21572468e-02, 9.87842753e-01],
       [7.82149051e-05, 9.99921785e-01],
       ...,
       [1.00000000e+00, 1.55630463e-24],
       [1.00000000e+00, 7.79956307e-19],
       [1.00000000e+00, 3.84435383e-21]])

In [554]:
predict_result = pd.concat([data_2022,pd.DataFrame(lr.predict_proba(x_2022.values))],axis=1)[['Player','position',0,1]].sort_values(by=1,ascending=False)

In [555]:
predict_result.sort_values(by=1,ascending=False).head(20)

,Player,position,0,1
2,Giannis Antetokounmpo,F,0.000078,0.999922
9,Nikola Jokic,C,0.000128,0.999872
0,Joel Embiid,C-F,0.000966,0.999034
3,Kevin Durant,F,0.003690,0.996310
96,Chris Paul,G,0.004564,0.995436
5,Trae Young,G,0.010756,0.989244
1,LeBron James,F,0.012157,0.987843
11,Devin Booker,G,0.013023,0.986977
4,Luka Doncic,F-G,0.020762,0.979238
10,Jayson Tatum,F-G,0.031975,0.968025


In [281]:
data_2022.position.unique()

array(['C-F', 'F', 'F-G', 'G', 'G-F', 'C', 'F-C'], dtype=object)

In [580]:
data_2022 = pd.read_csv(r'/Users/andyhsu/Desktop/NBA年度前三隊預測/球員資料/2022球員資料.csv',encoding='utf-8-sig')
for x,y in predict_result.groupby(['position']):
    if x == 'G':
        pos_df      = y[['Player',1]]
        player_list = y.Player.head(10)
        print(player_list)
        pd.set_option('display.max_columns',None)
        particular_perform = data_2022[data_2022['Player'].isin(player_list)][['Player', 'Team', 'AGE', 'GP', 'W', 'L', 'MIN', 'PTS', 'FGA', 'FG%',
        '3PA', '3P%', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL',
        'BLK', '累積前三隊', '+/-', '勝率', 'DFG_percent', 'DIFF_percent']]
        final_df    = particular_perform.merge(pos_df,on='Player',how='right')
        print(pd.DataFrame(final_df).dropna(how='any',axis=0))

96          Chris Paul
5           Trae Young
11        Devin Booker
13       Stephen Curry
26        James Harden
8            Ja Morant
50        Jrue Holiday
27      Darius Garland
12    Donovan Mitchell
33       Fred VanVleet
Name: Player, dtype: object
             Player Team   AGE    GP     W     L   MIN   PTS   FGA   FG%  \
0        Chris Paul  PHX  36.0  65.0  53.0  12.0  32.9  14.7  11.3  49.3   
1        Trae Young  ATL  23.0  76.0  40.0  36.0  34.9  28.4  20.3  46.0   
2      Devin Booker  PHX  25.0  68.0  56.0  12.0  34.5  26.8  20.9  46.6   
3     Stephen Curry  GSW  34.0  64.0  45.0  19.0  34.5  25.5  19.1  43.7   
4      James Harden  PHI  32.0  65.0  40.0  25.0  37.2  22.0  15.3  41.0   
5         Ja Morant  MEM  22.0  57.0  36.0  21.0  33.1  27.4  20.6  49.3   
6      Jrue Holiday  MIL  31.0  67.0  47.0  20.0  32.9  18.3  14.2  50.1   
7    Darius Garland  CLE  22.0  68.0  37.0  31.0  35.7  21.7  17.3  46.2   
8  Donovan Mitchell  UTA  25.0  67.0  41.0  26.0  33.8  25